In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import normalize

2023-10-30 21:06:39.191422: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
def standardize(arr):
    arr[0] = (abs(arr[0]) - 3.870671003)/1.899821718
    arr[1] = (abs(arr[1]) - 28.63948643)/12.58555761
    arr[2] = (abs(arr[2]) - 5.428999742)/2.474173139
    arr[3] = (abs(arr[3]) - 1.09667515)/0.473910857
    arr[4] = (abs(arr[4]) - 1425.476744)/1132.462122
    arr[5] = (abs(arr[5]) - 3.070655159)/10.38604956
    arr[6] = (abs(arr[6]) - 35.63186143)/2.135952397
    arr[7] = (abs(arr[7]) - 119.5697045)/2.003531724
        

In [21]:
#import dataset
raw = fetch_california_housing(as_frame=True)

#get input and output
X = raw.data.values
np.apply_along_axis(standardize,axis=1,arr=X)
Y = raw.target.values
n = len(X)
r = 0.25
x_train, x_test, y_train, y_test = train_test_split(X, Y, shuffle=True, test_size=r)

x_val = x_train[:int(n*r)]
part_x_train = x_train[int(n*r):]
y_val = y_train[:int(n*r)]
part_y_train = y_train[int(n*r):]

In [ ]:
from MultiLevelLayer import MultiLevel

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.InputLayer(input_shape=(8,)))
model.add(tf.keras.layers.Dropout(0))
model.add(tf.keras.layers.Dense(32,activation='sigmoid'))
model.add(tf.keras.layers.Dropout(.20))
# model.add(MultiLevel(2,False))
model.add(tf.keras.layers.Dense(16,activation='sigmoid'))
model.add(tf.keras.layers.Dropout(.20))
model.add(tf.keras.layers.Dense(1,activation='relu'))
tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(loss='mean_squared_error',optimizer='sgd')

historyBefore = model.fit(part_x_train,
                    part_y_train,validation_data=(x_val,y_val),
                    epochs=200,
                    batch_size=1
                    )

Epoch 1/200
10320/10320 [==============================] - 23s 2ms/step - loss: 0.8008 - val_loss: 0.5502
Epoch 2/200
10320/10320 [==============================] - 23s 2ms/step - loss: 0.6028 - val_loss: 0.5067
Epoch 3/200
10320/10320 [==============================] - 22s 2ms/step - loss: 0.5660 - val_loss: 0.4684
Epoch 4/200
10320/10320 [==============================] - 20s 2ms/step - loss: 0.5388 - val_loss: 0.4655
Epoch 5/200
10320/10320 [==============================] - 21s 2ms/step - loss: 0.5265 - val_loss: 0.4382
Epoch 6/200
10320/10320 [==============================] - 23s 2ms/step - loss: 0.5183 - val_loss: 0.4457
Epoch 7/200
10320/10320 [==============================] - 25s 2ms/step - loss: 0.5097 - val_loss: 0.4568
Epoch 8/200
10320/10320 [==============================] - 22s 2ms/step - loss: 0.5027 - val_loss: 0.4243
Epoch 9/200
10320/10320 [==============================] - 22s 2ms/step - loss: 0.4954 - val_loss: 0.4280
Epoch 10/200
10320/10320 [====================

In [ ]:
thing = model.predict(x_test)

In [ ]:
plt.scatter(thing,y_test,s=2)
plt.title('Median House Value')
plt.xlabel('Predicted Value (100 thousand Dollars)')
plt.ylabel('Actual Value (100 thousand Dollars)')
x_temp=[-.25,5.5]
y_temp=[-0.25,5.5]
plt.plot(x_temp, y_temp, 'r')
plt.show()

In [ ]:
scalar =2
use_all_nodes = False
level_size=2

arr = np.full(32, 0, dtype='float32')
for i in range(0, len(arr), level_size):
    temp = tf.random.uniform(shape=(), minval=0, maxval=level_size,dtype='int32')
    arr[i + temp] = scalar if not use_all_nodes else 1

print(arr)

In [ ]:
class SliceMasker:
    def __init__(self, num_samples, num_features, step_size):
        self.pseudo_logits = tf.ones(shape=(1, step_size))
        self.num_masked = num_samples//step_size
        self.offsets = tf.expand_dims(
            tf.range(self.num_masked) * step_size, axis=0)
        self.mask = tf.zeros((self.num_masked, num_features))

    def mask_slices(self, tensor):
        slice_ids = tf.random.categorical(
            self.pseudo_logits, self.num_masked, dtype=tf.int32)
        slice_ids = tf.reshape(slice_ids + self.offsets, (self.num_masked, 1))
        return tf.tensor_scatter_nd_update(tensor, slice_ids, self.mask)


num_samples = 21
num_features = 1
step_size = 3

sm = SliceMasker(num_samples, num_features, step_size)


example = tf.random.normal(shape=(num_samples, num_features))

print("before masking:")
print(example.numpy())
masked = sm.mask_slices(example)
print("after masking:")
print(masked.numpy())
